In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import spacy
from spacy.matcher import PhraseMatcher
from ipynb.fs.full.helper import get_text_from_csv,read_fb,clean_text,check_null,find_words,find_topic,get_word_lable,count_topic,count_word


# GOOGLE + FACEBOOK

In [2]:
google_df = get_text_from_csv('GoogleAds/CT.csv')
google_df

,text,advertiser_name,platform
0,"""Our Democrat opponent outraised us last quart...","Team Herschel, Inc.",google
1,"""Our Democrat opponent outraised us last quart...","Team Herschel, Inc.",google
2,"""It's time to focus on the hard-working famili...",GEORGE LOGAN FOR CONGRESS,google
3,"""Our Democrat opponent outraised us last quart...","Team Herschel, Inc.",google
4,"""Our Democrat opponent outraised us last quart...","Team Herschel, Inc.",google
5,"""Our Democrat opponent outraised us last quart...","Team Herschel, Inc.",google
6,"""Help Herschel Walker reach his critical end o...","Team Herschel, Inc.",google
7,"""Georgia will decide the Senate Majority in 20...","Team Herschel, Inc.",google
8,"""Tell Gov. Lamont to stop putting innovation a...",PhRMA,google
9,"""Gov. Lamont is touting a proposal that would ...",PhRMA,google


In [3]:
facebook_df = read_fb('FacebookAds/Connecticut.csv')
facebook_df

,text,advertiser_name,platform
0,"Please join CEA's Red for Ed Day of Action this Wednesday! Come to Hartford to meet with your legislators, or if you can't make it, take just a few minutes to call or email.",CONNECTICUT EDUCATION ASSOCIATION,facebook
1,"Thinking about attending a Connecticut State Community College? Get all the information and support you need during our virtual showcases April 28-30th. Hear about community college and program options throughout the state, transfer opportunities, financial aid, careers and more all from your couch! Live chat will be available. Register today by visiting the link in our bio.",odonnell company,facebook
2,Governor Lamont is protecting abortion rights in Connecticut. The governor is set to sign a major piece of legislation protecting abortion ahead of an upcoming Supreme Court ruling. Sign up now to learn more about what the Governor is doing for Connecticut >>,NED FOR CT,facebook
3,"Hey Connecticut, have you heard? Governor Lamont has signed a bill suspending the gas tax. This means real relief for working families across the state. Add your name now to get more updates on what Gov. Lamont is doing for Connecticut >>",NED FOR CT,facebook
4,Cost increases are hitting families in Connecticut. That’s why Governor Lamont and the Legislature have suspended the gas tax. It’ll cost you less to fill up at the pump. That’s real relief for Connecticut families. Add your name and find out more about how the Governor is working for you.,NED FOR CT,facebook
...,...,...,...
1280,"Connecticut Dairy farmers understand the importance of thoughtful, yet vigorous work to increase sustainability on the farm. Watch and learn how two local farms are tackling climate change with soil management practices. 🌱",NaN,facebook
1281,"Connecticut Dairy farmers understand the importance of thoughtful, yet vigorous work to increase sustainability on the farm. Watch and learn how two local farms are tackling climate change with soil management practices. 🌱",NaN,facebook
1282,"Connecticut Dairy farmers understand the importance of thoughtful, yet vigorous work to increase sustainability on the farm. Watch and learn how two local farms are tackling climate change with soil management practices. 🌱",NaN,facebook
1283,"Connecticut Dairy farmers understand the importance of thoughtful, yet vigorous work to increase sustainability on the farm. Watch and learn how two local farms are tackling climate change with soil management practices. 🌱",NaN,facebook


In [4]:
# cncat google_df and facebook_df by 'text'
df_text = pd.concat([google_df,facebook_df])
df_text

,text,advertiser_name,platform
0,"""Our Democrat opponent outraised us last quarter, so we need your help immediately."",""The Georgia polls are a dead heat, so please help us close the fundraising gap today!","Team Herschel, Inc.",google
1,"""Our Democrat opponent outraised us last quarter, so we need your help immediately."",""Georgia will decide the Senate Majority in 2022, so help us close the fundraising gap!","Team Herschel, Inc.",google
2,"""It's time to focus on the hard-working families in Connecticut and end the D.C. power grab"",""George Logan is the Republican leadership Washington and Connecticut need. Donate today!",GEORGE LOGAN FOR CONGRESS,google
3,"""Our Democrat opponent outraised us last quarter, so we need your help immediately."",""Georgia will decide the Senate Majority in 2022, so help us close the fundraising gap!","Team Herschel, Inc.",google
4,"""Our Democrat opponent outraised us last quarter, so we need your help immediately."",""Georgia will decide the Senate Majority in 2022, so help us close the fundraising gap!","Team Herschel, Inc.",google
...,...,...,...
1280,"Connecticut Dairy farmers understand the importance of thoughtful, yet vigorous work to increase sustainability on the farm. Watch and learn how two local farms are tackling climate change with soil management practices. 🌱",NaN,facebook
1281,"Connecticut Dairy farmers understand the importance of thoughtful, yet vigorous work to increase sustainability on the farm. Watch and learn how two local farms are tackling climate change with soil management practices. 🌱",NaN,facebook
1282,"Connecticut Dairy farmers understand the importance of thoughtful, yet vigorous work to increase sustainability on the farm. Watch and learn how two local farms are tackling climate change with soil management practices. 🌱",NaN,facebook
1283,"Connecticut Dairy farmers understand the importance of thoughtful, yet vigorous work to increase sustainability on the farm. Watch and learn how two local farms are tackling climate change with soil management practices. 🌱",NaN,facebook


In [5]:
df_text = clean_text(df_text)
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             233 non-null    object
 1   advertiser_name  225 non-null    object
 2   platform         233 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


In [6]:
df_text.head()

,text,advertiser_name,platform
0,our democrat opponent outraised us last quarter so we need your help immediately the georgia polls are a dead heat so please help us close the fundraising gap today,"Team Herschel, Inc.",google
1,our democrat opponent outraised us last quarter so we need your help immediately georgia will decide the senate majority in 2022 so help us close the fundraising gap,"Team Herschel, Inc.",google
2,it s time to focus on the hard working families in connecticut and end the d c power grab george logan is the republican leadership washington and connecticut need donate today,GEORGE LOGAN FOR CONGRESS,google
3,help herschel walker reach his critical end of quarter fundraising goal donate to the official website of herschel walker for united states senate today,"Team Herschel, Inc.",google
4,georgia will decide the senate majority in 2022 so help us close the fundraising gap our democrat opponent outraised us last quarter so we need your help immediately,"Team Herschel, Inc.",google


In [7]:
df_text = get_word_lable(df_text,'lexicon.json')
df_text

,text,advertiser_name,platform,words,m_label
0,our democrat opponent outraised us last quarter so we need your help immediately the georgia polls are a dead heat so please help us close the fundraising gap today,"Team Herschel, Inc.",google,no words,no topic
1,our democrat opponent outraised us last quarter so we need your help immediately georgia will decide the senate majority in 2022 so help us close the fundraising gap,"Team Herschel, Inc.",google,no words,no topic
2,it s time to focus on the hard working families in connecticut and end the d c power grab george logan is the republican leadership washington and connecticut need donate today,GEORGE LOGAN FOR CONGRESS,google,donate,donation
3,help herschel walker reach his critical end of quarter fundraising goal donate to the official website of herschel walker for united states senate today,"Team Herschel, Inc.",google,donate,donation
4,georgia will decide the senate majority in 2022 so help us close the fundraising gap our democrat opponent outraised us last quarter so we need your help immediately,"Team Herschel, Inc.",google,no words,no topic
...,...,...,...,...,...
228,drugs flood through the border killing our families and neighbors sign to save our families,Connecticut Republican Party,facebook,"drugs,border","immigration,safety"
229,last tuesday i had the privilege of appearing on fox 94 9 lee elci the voice of freedom here s a video in case you missed it,Robert Boris for State Representative,facebook,no words,no topic
230,this is it the biggest test of my career in public service tonight we wrap up our fundraising period democrats across the state will use this information when deciding who to support at the critical state convention will our fundraising finally convince the establishment that this campaign is for real you can help answer that question by contributing now hilda,Santiago for Connecticut,facebook,no words,no topic
231,ctteacher laura baker coronis got an unexpected surprise today when milken educator awards senior vice president jane foley called her name as this season s ct milken educator award recipient the ansonia high school community went wild to celebrate the math teacher ansonia public schools connecticut state department of education office of lt governor susan bysiewicz educatect ansonia ansoniact goodnews,MILKEN FAMILY FOUNDATION,facebook,"state department,school,public schools,education","education,foreign policy"


In [26]:
df_text.to_csv('results/ad_ct.csv')

In [9]:
text_topic = count_topic(df_text)
text_topic

,single_topic,counts
0,no topic,102
1,economic,41
2,education,30
3,governance,28
4,health,25
5,social programs,18
6,social and cultural,14
7,environment,12
8,safety,11
9,covid,8


In [10]:
df_text[df_text['m_label']=='no topic']

,text,advertiser_name,platform,words,m_label
0,our democrat opponent outraised us last quarter so we need your help immediately the georgia polls are a dead heat so please help us close the fundraising gap today,"Team Herschel, Inc.",google,no words,no topic
1,our democrat opponent outraised us last quarter so we need your help immediately georgia will decide the senate majority in 2022 so help us close the fundraising gap,"Team Herschel, Inc.",google,no words,no topic
4,georgia will decide the senate majority in 2022 so help us close the fundraising gap our democrat opponent outraised us last quarter so we need your help immediately,"Team Herschel, Inc.",google,no words,no topic
5,tell gov lamont to stop putting innovation and access at risk send a letter here protect live saving research from gov lamont s government price setting proposal,PhRMA,google,no words,no topic
7,there s no greater priority than defending democracy you can help,Democracy Initiative,google,no words,no topic
...,...,...,...,...,...
225,ned lamont radical dems are letting criminals loose act to keep dangerous criminals off the streets,Connecticut Republican Party,facebook,no words,no topic
226,richard blumenthal is failing the people of connecticut sign today to fire blumenthal in 2022,Connecticut Republican Party,facebook,no words,no topic
227,sign the petition today if you want to replace richard blumenthal in 2022 act now,Connecticut Republican Party,facebook,no words,no topic
229,last tuesday i had the privilege of appearing on fox 94 9 lee elci the voice of freedom here s a video in case you missed it,Robert Boris for State Representative,facebook,no words,no topic


In [11]:
text_word = count_word(df_text)
text_word

,single_word,counts
0,no words,102
1,education,17
2,health,15
3,tax,15
4,public schools,13
...,...,...
125,natural gas,1
126,deductibles,1
127,deficit,1
128,loan assistance,1


In [12]:
df_text['advertiser_name'].value_counts()

School and State Finance Project                                            25
NED FOR CT                                                                  18
Connecticut Republican Party                                                11
50CAN, INC.                                                                 10
Bob for Governor - David Becker, Treasurer - Approved by Bob Stefanowski     9
                                                                            ..
Andrew Adam Theriac                                                          1
Financial Study Network                                                      1
CT State Employee Bargaining Agent Coalition                                 1
Jacobs & Jacobs                                                              1
Lumaj for Senate                                                             1
Name: advertiser_name, Length: 68, dtype: int64

# GOOGLE

In [13]:
if google_df['text'].isnull().sum():
    print (google_df['text'].isnull().sum())
    google_df = google_df[google_df['text'].notnull()]
    google_df.reset_index(drop=True,inplace=True)

In [14]:
google_df = clean_text(google_df)
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             28 non-null     object
 1   advertiser_name  28 non-null     object
 2   platform         28 non-null     object
dtypes: object(3)
memory usage: 800.0+ bytes


In [15]:
google_df = get_word_lable(google_df,'lexicon.json')
google_df.head()

,text,advertiser_name,platform,words,m_label
0,our democrat opponent outraised us last quarter so we need your help immediately the georgia polls are a dead heat so please help us close the fundraising gap today,"Team Herschel, Inc.",google,no words,no topic
1,our democrat opponent outraised us last quarter so we need your help immediately georgia will decide the senate majority in 2022 so help us close the fundraising gap,"Team Herschel, Inc.",google,no words,no topic
2,it s time to focus on the hard working families in connecticut and end the d c power grab george logan is the republican leadership washington and connecticut need donate today,GEORGE LOGAN FOR CONGRESS,google,donate,donation
3,help herschel walker reach his critical end of quarter fundraising goal donate to the official website of herschel walker for united states senate today,"Team Herschel, Inc.",google,donate,donation
4,georgia will decide the senate majority in 2022 so help us close the fundraising gap our democrat opponent outraised us last quarter so we need your help immediately,"Team Herschel, Inc.",google,no words,no topic


In [16]:
google_topic = count_topic(google_df)
google_topic

,single_topic,counts
0,no topic,15
1,governance,6
2,economic,3
3,donation,2
4,social programs,2
5,covid,1
6,immigration,1


In [17]:
google_word = count_word(google_df)
google_word

,single_word,counts
0,no words,15
1,filibuster,4
2,donate,2
3,laws,2
4,border,1
5,covid,1
6,jobs,1
7,out of pocket,1
8,out of pocket costs,1
9,savings,1


# FACEBOOK 

In [18]:
if facebook_df['text'].isnull().sum():
    print (facebook_df['text'].isnull().sum())
    facebook_df = facebook_df[facebook_df['text'].notnull()]
    facebook_df.reset_index(drop=True,inplace=True)

5


In [19]:
facebook_df = clean_text(facebook_df)
facebook_df.info

/Users/yzhao/git/ml-political-ads/helper.ipynb:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "id": "5b928802",
/Users/yzhao/git/ml-political-ads/helper.ipynb:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "metadata": {},
/Users/yzhao/git/ml-political-ads/helper.ipynb:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

<bound method DataFrame.info of                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 text  \
0                                                                                                                                                                                                                                                                                                                                                               

In [20]:
facebook_df = get_word_lable(facebook_df,'lexicon.json')
facebook_df.head()

/Users/yzhao/git/ml-political-ads/helper.ipynb:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "    for t in topics:\n",
/Users/yzhao/git/ml-political-ads/helper.ipynb:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        matcher = PhraseMatcher(nlp.vocab)\n",
/Users/yzhao/git/ml-political-ads/helper.ipynb:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,text,advertiser_name,platform,words,m_label
0,please join cea s red for ed day of action this wednesday come to hartford to meet with your legislators or if you can t make it take just a few minutes to call or email,CONNECTICUT EDUCATION ASSOCIATION,facebook,no words,no topic
1,thinking about attending a connecticut state community college get all the information and support you need during our virtual showcases april 28 30th hear about community college and program options throughout the state transfer opportunities financial aid careers and more all from your couch live chat will be available register today by visiting the link in our bio,odonnell company,facebook,"college,community college",social programs
2,governor lamont is protecting abortion rights in connecticut the governor is set to sign a major piece of legislation protecting abortion ahead of an upcoming supreme court ruling sign up now to learn more about what the governor is doing for connecticut,NED FOR CT,facebook,"supreme court,abortion","governance,health,social and cultural"
3,hey connecticut have you heard governor lamont has signed a bill suspending the gas tax this means real relief for working families across the state add your name now to get more updates on what gov lamont is doing for connecticut,NED FOR CT,facebook,tax,economic
4,cost increases are hitting families in connecticut that s why governor lamont and the legislature have suspended the gas tax it ll cost you less to fill up at the pump that s real relief for connecticut families add your name and find out more about how the governor is working for you,NED FOR CT,facebook,tax,economic


In [21]:
facebook_topic = count_topic(facebook_df)
facebook_topic

,single_topic,counts
0,no topic,86
1,economic,38
2,education,30
3,health,25
4,governance,22
5,social programs,16
6,social and cultural,14
7,environment,12
8,safety,11
9,foreign policy,8


In [22]:
facebook_word = count_word(facebook_df)
facebook_word

,single_word,counts
0,no words,86
1,education,17
2,health,15
3,tax,14
4,public schools,13
...,...,...
121,loan assistance,1
122,living wage,1
123,covid 19,1
124,deductibles,1
